In [430]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats


In [431]:
path = "ExperimentData"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/ResponsiveAR/Experiment/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/ResponsiveAR/Experiment/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

Files and directories in ' ExperimentData ' :
['59', '32', '35', '56', '58', '34', '33', '.DS_Store', '36', '31', '30', '37', '39', '41', '49', '40']
['ExperimentData/59/ResponsiveAR/Experiment/', 'ExperimentData/32/ResponsiveAR/Experiment/', 'ExperimentData/35/ResponsiveAR/Experiment/', 'ExperimentData/56/ResponsiveAR/Experiment/', 'ExperimentData/58/ResponsiveAR/Experiment/', 'ExperimentData/34/ResponsiveAR/Experiment/', 'ExperimentData/33/ResponsiveAR/Experiment/', 'ExperimentData/36/ResponsiveAR/Experiment/', 'ExperimentData/31/ResponsiveAR/Experiment/', 'ExperimentData/30/ResponsiveAR/Experiment/', 'ExperimentData/37/ResponsiveAR/Experiment/', 'ExperimentData/39/ResponsiveAR/Experiment/', 'ExperimentData/41/ResponsiveAR/Experiment/', 'ExperimentData/49/ResponsiveAR/Experiment/', 'ExperimentData/40/ResponsiveAR/Experiment/']


In [432]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [433]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [434]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


ExperimentSession_0_1652910130460.json
ExperimentSession_1_1652910130460.json
ExperimentSession_2_1652910130460.json
ExperimentSession_3_1652910130460.json
ExperimentSession_4_1652910130460.json
ExperimentSession_5_1652910130460.json
ExperimentSession_6_1652910130460.json
ExperimentSession_7_1652910130460.json
ExperimentSession_8_1652910130460.json
ExperimentSession_9_1652910130460.json
ExperimentSession_0_1652736949521.json
ExperimentSession_1_1652736949521.json
ExperimentSession_2_1652736949521.json
ExperimentSession_3_1652736949521.json
ExperimentSession_4_1652736949521.json
ExperimentSession_5_1652736949521.json
ExperimentSession_6_1652736949521.json
ExperimentSession_7_1652736949521.json
ExperimentSession_8_1652736949521.json
ExperimentSession_9_1652736949521.json
ExperimentSession_0_1652807270806.json
ExperimentSession_1_1652807270806.json
ExperimentSession_2_1652807270806.json
ExperimentSession_3_1652807270806.json
ExperimentSession_4_1652807270806.json
ExperimentSession_5_16528

In [435]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] < 9 ]
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,59,137,The task is to click on the 2nd article title....,1652910156140,1652910160750,4610,0,True,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165..."


USER 56 was unable to finish task 7 (when starting from 0) remove their task 8 from data

In [436]:
session_to_drop = sessionsDf.loc[(sessionsDf["userID"] == 56) & (sessionsDf["sessionNumber"] == 7)]
sessionsDf.drop(session_to_drop.index)

session_to_drop

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
37,56,1434,The task is to find the current location on th...,1652899626599,1652899774967,148368,7,False,"[{'userID': 56, 'frameNum': 1741, 'unixTime': ..."


In [437]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf.head(1)

(48370, 14)


,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,59,1,1652910146930,16470,"{'x': -0.01730290986597538, 'y': -0.0262664388...","{'x': 0.0704965889453888, 'y': -0.144727870821...","{'x': 0.07085785269737244, 'y': 0.038738552480...","{'x': 8.321520805358887, 'y': 4.08559703826904...","{'x': -0.016758747398853302, 'y': -0.026269029...","{'x': 0.07254093885421753, 'y': -0.14517901837...","{'x': -0.04117072373628616, 'y': 1.03524494171...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1652910146830, 'systemTime': '14...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [438]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
print(users.shape)

experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)
experimentDf.shape

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID

experimentDf.head(1)

(807, 14)
(1084, 10)


,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1652910146830,14-42-26-83,True,instruction,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.015964265912771225, 'y': 0.0036757867...",2nd Title,,59.0


In [439]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
1,1652910156190,14-42-36-19,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.015964265912771225, 'y': 0.0036757867...",2nd Title,,59.0
3,1652743234566,16-20-34-56,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.010610290803015232, 'y': 0.01864774152...",2nd Title,,33.0
5,1652726010089,11-33-30-08,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.17531268298625946, 'y': 0.091279007494...",2nd Title,,31.0


In [440]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,59,137,The task is to click on the 2nd article title....,1652910156140,1652910160750,4610,0,True,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165..."
60,33,94,The task is to click on the 2nd article title....,1652743234506,1652743239166,4660,0,True,"[{'userID': 33, 'frameNum': 1, 'unixTime': 165..."
80,31,188,The task is to click on the 2nd article title....,1652726010029,1652726017089,7060,0,True,"[{'userID': 31, 'frameNum': 1, 'unixTime': 165..."
50,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165..."
90,30,240,The task is to click on the 2nd article title....,1652721816735,1652721833975,17240,0,False,"[{'userID': 30, 'frameNum': 1, 'unixTime': 165..."
...,...,...,...,...,...,...,...,...,...
38,56,589,The task is to find the low temperature on th...,1652899798007,1652899836138,38131,8,False,"[{'userID': 56, 'frameNum': 3175, 'unixTime': ..."
58,34,1464,The task is to find the low temperature on th...,1652747802011,1652747952843,150832,8,False,"[{'userID': 34, 'frameNum': 2625, 'unixTime': ..."
88,31,164,The task is to find the low temperature on th...,1652726553097,1652726562238,9141,8,True,"[{'userID': 31, 'frameNum': 4994, 'unixTime': ..."
98,30,219,The task is to find the low temperature on th...,1652722052359,1652722061732,9373,8,False,"[{'userID': 30, 'frameNum': 2253, 'unixTime': ..."


In [441]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
0,59,137,The task is to click on the 2nd article title....,1652910156140,1652910160750,4610,0,True,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1652910156190


In [442]:
endEvents = experimentDf.loc[experimentDf["eventName"] == "complete"]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
32,1652818008923,13-06-48-92,True,complete,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.045453257858753204, 'y': 0.06533057987...",2nd Title,2nd Title,37.0


In [443]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time
0,59,137,The task is to click on the 2nd article title....,1652910156140,1652910160750,4610,0,True,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1652910156190,1652910160750


In [444]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([  0,   0, -10, -10, -10,   0,   0,   0,   0,   0, -11,   0, -10,
       -11,  -5,   0,   0,   0,   0,   0, -10, -10, -10,   0, -10,   0,
         0, -10,  -2,   0, -10,   0,  -9, -11,  -9,  -6,  -4,   0,   0,
         0,   0, -10,   0,   0,   0, -10, -10, -10, -11, -10,   0, -10,
        -7,  -9,   0, -10,   0,   0, -10,  -9,   0,  -5,   0,   0,   0,
         0,   0, -10,  -7,  -5,   0,   0,   0, -11,  -9,   0,  -9,   0,
        -5,  -9, -10,   0,   0,   0,   0,   0,   0,   0,   0,   0, -10,
         0,   0,   0,  -9,  -5, -10,   0, -10,   0,   0,   0,   0,  -4,
         0,   0,  -9,   0,   0,   0,   0, -10, -10,   0,   0, -10,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0])

In [445]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([-50, -60, -60, -60, -59, -50, -50, -50, -45, -50, -50, -50, -50,
       -56, -60, -40, -40, -40, -39, -46, -40, -40, -28, -39, -50, -45,
       -40, -30, -40, -40, -41, -40, -40, -40, -40, -41, -30, -40, -39,
       -41, -41, -40, -39, -39, -40, -40, -39, -40, -50, -40, -40, -40,
       -41, -40, -40, -39, -41, -40, -40, -46, -20, -21, -21, -19, -20,
       -20, -20, -20, -16, -10, -20, -10, -19, -20, -21,  -4, -10, -10,
       -10, -10,   0,  -4, -10,   0, -10, -10, -10,  -6,  -3, -10,  -8,
       -20, -10, -10, -10, -10, -19, -16, -10, -20,  -9, -20, -11, -20,
        -4,   0, -10,   0, -10, -10, -10, -10, -10, -10,  -8,   0,  -5,
       -10,   0,   0, -10,   0, -10, -10, -10, -13, -10, -10, -10,  -5,
       -10, -10, -10,   0, -16])

In [446]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
0,59,137,The task is to click on the 2nd article title....,1652910156140,1652910160750,4610,0,True,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1652910156190,1652910160750,4560


In [447]:
start_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
start_time_diff

array([ 50,  60,  50,  50,  49,  50,  50,  50,  45,  50,  39,  50,  40,
        45,  55,  40,  40,  40,  39,  46,  30,  30,  18,  39,  40,  45,
        40,  20,  38,  40,  31,  40,  31,  29,  31,  35,  26,  40,  39,
        41,  41,  30,  39,  39,  40,  30,  29,  30,  39,  30,  40,  30,
        34,  31,  40,  29,  41,  40,  30,  37,  20,  16,  21,  19,  20,
        20,  20,  10,   9,   5,  20,  10,  19,   9,  12,   4,   1,  10,
         5,   1, -10,   4,  10,   0,  10,  10,  10,   6,   3,  10,  -2,
        20,  10,  10,   1,   5,   9,  16,   0,  20,   9,  20,  11,  16,
         4,   0,   1,   0,  10,  10,  10,   0,   0,  10,   8, -10,   5,
        10,   0,   0,  10,   0,  10,  10,  10,  13,  10,  10,  10,   5,
        10,  10,  10,   0,  16])

In [448]:
responsiveDf = sessionsDf.loc[sessionsDf["isResponsive"] == True]
responsiveDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
0,59,137,The task is to click on the 2nd article title....,1652910156140,1652910160750,4610,0,True,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1652910156190,1652910160750,4560
60,33,94,The task is to click on the 2nd article title....,1652743234506,1652743239166,4660,0,True,"[{'userID': 33, 'frameNum': 1, 'unixTime': 165...",1652743234566,1652743239166,4600
80,31,188,The task is to click on the 2nd article title....,1652726010029,1652726017089,7060,0,True,"[{'userID': 31, 'frameNum': 1, 'unixTime': 165...",1652726010089,1652726017099,7010
100,37,52,The task is to click on the 2nd article title....,1652818005343,1652818008923,3580,0,True,"[{'userID': 37, 'frameNum': 1, 'unixTime': 165...",1652818005393,1652818008923,3530
110,39,220,The task is to click on the 2nd article title....,1652829254126,1652829262856,8730,0,True,"[{'userID': 39, 'frameNum': 1, 'unixTime': 165...",1652829254176,1652829262856,8680
...,...,...,...,...,...,...,...,...,...,...,...,...
68,33,604,The task is to find the low temperature on th...,1652743405209,1652743460479,55270,8,True,"[{'userID': 33, 'frameNum': 1626, 'unixTime': ...",1652743405219,1652743460479,55260
108,37,123,The task is to find the low temperature on th...,1652818570763,1652818580223,9460,8,True,"[{'userID': 37, 'frameNum': 4619, 'unixTime': ...",1652818570773,1652818580223,9450
118,39,1429,The task is to find the low temperature on th...,1652829423309,1652829574206,150897,8,True,"[{'userID': 39, 'frameNum': 1712, 'unixTime': ...",1652829423319,1652829574206,150887
28,35,256,The task is to find the low temperature on th...,1652807609202,1652807623642,14440,8,True,"[{'userID': 35, 'frameNum': 2989, 'unixTime': ...",1652807609212,1652807623642,14430


In [449]:
nonresponsiveDf = sessionsDf.loc[sessionsDf["isResponsive"] == False]
nonresponsiveDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
50,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165...",1652747526727,1652747540617,13890
90,30,240,The task is to click on the 2nd article title....,1652721816735,1652721833975,17240,0,False,"[{'userID': 30, 'frameNum': 1, 'unixTime': 165...",1652721816794,1652721833985,17191
40,58,121,The task is to click on the 2nd article title....,1652894765461,1652894769851,4390,0,False,"[{'userID': 58, 'frameNum': 1, 'unixTime': 165...",1652894765511,1652894769851,4340
30,56,121,The task is to click on the 2nd article title....,1652899449516,1652899453671,4155,0,False,"[{'userID': 56, 'frameNum': 1, 'unixTime': 165...",1652899449561,1652899453671,4110
70,36,120,The task is to click on the 2nd article title....,1652812829574,1652812832864,3290,0,False,"[{'userID': 36, 'frameNum': 1, 'unixTime': 165...",1652812829624,1652812832874,3250
...,...,...,...,...,...,...,...,...,...,...,...,...
78,36,174,The task is to find the low temperature on th...,1652812971891,1652812982716,10825,8,False,"[{'userID': 36, 'frameNum': 1424, 'unixTime': ...",1652812971896,1652812982716,10820
38,56,589,The task is to find the low temperature on th...,1652899798007,1652899836138,38131,8,False,"[{'userID': 56, 'frameNum': 3175, 'unixTime': ...",1652899798017,1652899836138,38121
58,34,1464,The task is to find the low temperature on th...,1652747802011,1652747952843,150832,8,False,"[{'userID': 34, 'frameNum': 2625, 'unixTime': ...",1652747802021,1652747952843,150822
98,30,219,The task is to find the low temperature on th...,1652722052359,1652722061732,9373,8,False,"[{'userID': 30, 'frameNum': 2253, 'unixTime': ...",1652722052359,1652722061732,9373


In [450]:
#task 0
i = 0
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 0 Time: 5878.875 ± 2176.016
normal task 0 Time: 13410.143 ± 15863.556
Ttest_indResult(statistic=-1.3356626122704116, pvalue=0.20457399262882564)


In [451]:
#Task 1
i = 1
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 1 Time: 3638.875 ± 1767.838
normal task 1 Time: 19412.286 ± 15580.939
Ttest_indResult(statistic=-2.857849169491856, pvalue=0.013452981610666104)


In [452]:
#Task 2
i = 2
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 2 Time: 5770.250 ± 2524.089
normal task 2 Time: 11023.000 ± 6557.671
Ttest_indResult(statistic=-2.1035906503915367, pvalue=0.055449973008566376)


In [453]:
#task 3
i = 3
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 3 Time: 6403.875 ± 2282.216
normal task 3 Time: 12381.714 ± 6265.725
Ttest_indResult(statistic=-2.525035711726866, pvalue=0.02536166456981908)


In [454]:
#task 4
i = 4
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 4 Time: 26003.000 ± 5243.128
normal task 4 Time: 22168.286 ± 6353.845
Ttest_indResult(statistic=1.2813796198084593, pvalue=0.22244538443306083)


In [455]:
#task 5
i = 5
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 5 Time: 47695.250 ± 57194.484
normal task 5 Time: 25177.429 ± 9618.066
Ttest_indResult(statistic=1.0243362583119957, pvalue=0.32436052557235096)


In [456]:
#task 6
i = 6
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 6 Time: 80245.500 ± 136045.714
normal task 6 Time: 19032.857 ± 6382.327
Ttest_indResult(statistic=1.1836357037369147, pvalue=0.2577498145764722)


In [457]:
#task 7
i = 7
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 7 Time: 76116.375 ± 40771.003
normal task 7 Time: 92505.857 ± 113606.862
Ttest_indResult(statistic=-0.382566805489794, pvalue=0.7082226501542859)


In [458]:
#task 8
i = 8
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 8 Time: 37675.000 ± 48505.706
normal task 8 Time: 34585.000 ± 52302.732
Ttest_indResult(statistic=0.11871136511198505, pvalue=0.9073187923959065)


In [459]:
#each task in a loop
for i in range(0,9):
    print(f"TASK {str(i)}:")
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["isResponsive"] == True]
    task_n = taskDf.loc[taskDf["isResponsive"] == False]

    taskRAvgTime = task_r['Actual_Total_Time'].mean()
    taskRStd = task_r['Actual_Total_Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Actual_Total_Time'].mean()
    taskNStd = task_n['Actual_Total_Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 0:
responsive task 0 Time: 5878.875 ± 2176.016
normal task 0 Time: 13410.143 ± 15863.556
Ttest_indResult(statistic=-1.3356626122704116, pvalue=0.20457399262882564)
TASK 1:
responsive task 1 Time: 3638.875 ± 1767.838
normal task 1 Time: 19412.286 ± 15580.939
Ttest_indResult(statistic=-2.857849169491856, pvalue=0.013452981610666104)
TASK 2:
responsive task 2 Time: 5770.250 ± 2524.089
normal task 2 Time: 11023.000 ± 6557.671
Ttest_indResult(statistic=-2.1035906503915367, pvalue=0.055449973008566376)
TASK 3:
responsive task 3 Time: 6403.875 ± 2282.216
normal task 3 Time: 12381.714 ± 6265.725
Ttest_indResult(statistic=-2.525035711726866, pvalue=0.02536166456981908)
TASK 4:
responsive task 4 Time: 26003.000 ± 5243.128
normal task 4 Time: 22168.286 ± 6353.845
Ttest_indResult(statistic=1.2813796198084593, pvalue=0.22244538443306083)
TASK 5:
responsive task 5 Time: 47695.250 ± 57194.484
normal task 5 Time: 25177.429 ± 9618.066
Ttest_indResult(statistic=1.0243362583119957, pvalue=0.32436052